# Embeddings


- Assumes user interest in movies can be roughly explained by d aspects.
- Each movie becomes a d-dimensional point where the value in dimension d represents how much the movie fits that aspects
- Embeddings can be learned from data
- No separate training process needed -- the embedding layer is just a hidden layer without one unit per dimension
- Supervised information (e.g., users watched the same two movies) trailors the learned embeddings for the desired task
- Intuitively the hidden units discover how to organize the items in the d-dimensional space in a way to best optimize the final objective

 **"Model (Prediction Function)."**
 
## Sparse matrix

For audience - movie like matrix, there might be a millon+ audiences with 10K+ movies, then most of the cells in such a matrix is empty.

To model it up more efficiently, we are going to have 2 phases.

### Make a dictionary

- A dictionary is just a mapping from each feature, in this case each movie, to an integer from 0 to the number of movies-1, i.e, `let movies:[String] = []` and `let dic = [0 ..< movies.count]`

### Dimensions

dimensions is about sqrt(sqrt(possible values)) - empirical rule-of-thumb

### Bucketize && Cross Columns

like a cell in a chessboard: (assuming longitude and latitude had already been bucketized into 10x )

```
 # YOUR CODE HERE: Make a feature column for the long_x_lat feature cross
  long_x_lat = tf.feature_column.crossed_column(
  set([bucketized_longitude, bucketized_latitude]), hash_bucket_size=1000) 
```  

### Prevention of over-fitting

L2 - Penalizing the model complexity by evaluating w\*w, extra large weight should be avoided all the time, a better model should be with distributed weights around zero.

<img src="true-positives-and-true-negatives.png">漏报和误报</img>

### ROC

<img src="roc.png">每一点都是在某个阈值上的True Positives和误报率</img>

### AUC


- if we pick a random positive and a random negative, what's the probability my model ranks them in the correct order.

<img src="auc.png">Area under the ROC Curve</img>

- gives an aggregate measure of performance aggregated across all possible classification thresholds

## Prediction Bias

Logistic Regression predictions should be unbiased:

- average of predictions == average of observations

otherwise the model has some bias

We need to have buckets of data to make calibration be meaningful.
For example, if I am looking at flipping a coin, any given coin flip will either come up exactly heads or exactly tails, basically exactly 1 or exactly 0, but my probabilistic predictions will be 0.5 or 0.3 or some value in between 0 and 1. So it only make sense to compare those mean predictions to mean observations if I aggregate across a sufficiently large number of them.

<img src="bias.png">Calibration Plots Show Bucketed Bias</img>

Let's try calculating accuracy for the following model that classified 100 tumors as malignant (the positive class) or benign (the negative class):

&nbsp;|&nbsp;
------|--------
True Positive (TP):<br>Reality: Malignant<br>ML model predicted: Malignant<br>Number of TP results: 1|False Positive (FP):<br>Reality: Benign<br>ML model predicted: Malignant<br>Number of FP results: 1
False Negative (FN):<br>Reality: Malignant<br>ML model predicted: Benign<br>Number of FN results: 8|True Negative (TN):<br>Reality: Benign<br>ML model predicted: Benign<br>Number of TN results: 90
 
Accuracy comes out to 0.91, or 91% (91 correct predictions out of 100 total examples). That means our tumor classifier is doing a great job of identifying malignancies, right?

Actually, let's do a closer analysis of positives and negatives to gain more insight into our model's performance.

Of the 100 tumor examples, 91 are benign (90 TNs and 1 FP) and 9 are malignant (1 TP and 8 FNs).

Of the 91 benign tumors, the model correctly identifies 90 as benign. That's good. However, of the 9 malignant tumors, the model only correctly identifies 1 as malignant—a terrible outcome, as 8 out of 9 malignancies go undiagnosed!

While 91% accuracy may seem good at first glance, another tumor-classifier model that always predicts benign would achieve the exact same accuracy (91/100 correct predictions) on our examples. In other words, our model is no better than one that has zero predictive ability to distinguish malignant tumors from benign tumors.

<img src="precision-recall.png"></img>
<img src="roc-auc.png"></img>

One possible solution that works is to just train for **longer**, as long as we don't **overfit**.
**We can do this by increasing the number the steps, the batch size, or both.**
All metrics improve at the same time, so our loss metric is a good proxy for both AUC and accuracy.
Notice how it takes many, many more iterations just to squeeze a few more units of AUC. This commonly happens. But often even this small gain is worth the costs.

from:
```
linear_classifier = train_linear_classifier_model(
    learning_rate=0.000005,
    steps=500,
    batch_size=20,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)
```

to:

```
linear_classifier = train_linear_classifier_model(
    learning_rate=0.000003,
    steps=20000,
    batch_size=500,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

evaluation_metrics = linear_classifier.evaluate(input_fn=predict_validation_input_fn)

print("AUC on the validation set: %0.2f" % evaluation_metrics['auc'])
print("Accuracy on the validation set: %0.2f" % evaluation_metrics['accuracy'])
```

ROC curves typically feature true positive rate on the Y axis, and false positive rate on the X axis. **This means that the top left corner of the plot is the “ideal” point** - a false positive rate of zero, and a true positive rate of one. This is not very realistic, but it does mean that a **larger area under the curve (AUC)** is usually better.